## Boarding Passengers


In [3]:
import numpy as np
import matplotlib.pyplot as plt

def arrange_seats(N_seats):
    """
    Simulate a run of boarding passengers
    where the first passengers forget his/her boarding pass
    and chooses a seat at random.
    
    The remaining passengers will choose their seat (if available),
    if not they will choose a random empty seat.
    
    We want to calculate the probability that the last person
    to board gets their assigned seat.
    """
    
    # Number people 
    people = np.arange(0, N_seats)

    # randomly shuffle them
    np.random.shuffle(people)
    available_seats = list(people)
    first = people[0]
    first_chooses = np.random.choice(available_seats, size=1)[0]
#     print("Person #%d chooses Seat #%d" % (first, first_chooses))
    
    # Remove that choice from the list of available seats
    available_seats.remove(first_chooses)
    people_left = list(people).copy()
    people_left.remove(first)
#     print("Available seats: ", available_seats)
#     print("People left: ", people_left)
    
    # Board the remaining passengers except the last
    for person in people[1:-1]:
        
        # Their seat is empty. Occupy it
        if person in available_seats:
#             print("\nPerson #%d chooses Seat #%d" % (person, person))
            available_seats.remove(person)
            people_left.remove(person)
#             print("Available seats: ", available_seats)
#             print("People left: ", people_left)

        # Their seat is not empty. Choose a random one
        else:
            chooses = np.random.choice(available_seats, size=1)[0]
#             print("\nSeat #%d not available, Person #%d chooses Seat #%d" % (person, person, chooses))
            available_seats.remove(chooses)
            people_left.remove(person)
#             print("Available seats: ", available_seats)
#             print("People left: ", people_left)

    # Finally check if the last person's seat is free
    if people_left[0] == available_seats[0]:
        return 1
    else:
        return 0

Let's run this many times

In [7]:
N_seats = 64
N_trials = 1000
total = 0
for k in range(N_trials):
    total += arrange_seats(N_seats)
    
pcent = total / N_trials

print("%d / %d [%.2f per cent]" % (total, N_trials, pcent))

486 / 1000 [0.49 per cent]
